In [ ]:
import scipy.io
import pandas as pd
import pandas as pd
from Bio import pairwise2

In [ ]:
mat = scipy.io.loadmat('./dataset_3106.mat') 
mat = {k:v for k, v in mat.items() if k[0] != '_'}

In [ ]:
mat

In [ ]:
data = pd.DataFrame({k: pd.Series(v[0]) for k, v in mat.items()})

In [ ]:
data

In [ ]:
Y_3106_df = pd.DataFrame(mat['Y_3106'])
label_name_flat = [item[0][0] for item in mat['label_name']]
label_name_df = pd.DataFrame(label_name_flat, columns=['label_name'])
protein_list_flat = [item[0] for item in mat['protein_list'][0]]
protein_list_df = pd.DataFrame(protein_list_flat, columns=['ACC'])
sequence_3106_flat = [item[0][0] for item in mat['sequence_3106']]
sequence_3106_df = pd.DataFrame(sequence_3106_flat, columns=['sequence_3106'])


In [ ]:
Y_3106_df.columns = [label_name_df['label_name']]
Y_3106_df['sequence'] = sequence_3106_df['sequence_3106']
Y_3106_df['ACC'] = protein_list_df['ACC']

In [ ]:
Y_3106_df.columns

In [ ]:
Y_3106_df

In [ ]:
Y_3106_df.columns.shape[0]-5-2

In [ ]:
Y_3106_df.to_csv("./Y_3106.csv")

In [ ]:
default = pd.read_csv("../multisub_5_partitions_unique.csv")
y_3106_df = pd.read_csv("./Y_3106.csv")

In [ ]:
default.columns,y_3106_df.columns

In [ ]:
# Manually removing columns
columns_to_remove_in_y_3106 = [
    'Centrosome',
    'Cytoskeleton',
    'Endosome',
    'Microsome',
    'Synapse',
    'label_name'
]
y_3106_df = y_3106_df.drop(columns_to_remove_in_y_3106,axis=1)


In [ ]:
columns_to_rename_in_y_3106 = {
    # old name : new name
    'Endoplasmic-Reticulum' : 'Endoplasmic reticulum',
    'Golgi-Apparatus' : 'Golgi apparatus',
    'Lysosome' : 'Lysosome/Vacuole',
    'Plasma-Membrane' : 'Cell membrane',
}

y_3106_df = y_3106_df.rename(columns=columns_to_rename_in_y_3106)

In [ ]:
columns_to_add_in_y_3106 = {
    # column name : default value
    'Kingdom' : '',
    'Partition' : '',
    'Membrane' : 0.0,
    'Plastid' : 0.0,
}
for col in columns_to_add_in_y_3106:
    y_3106_df[col] = columns_to_add_in_y_3106[col]

In [ ]:
y_3106_df

In [ ]:
default =default.drop('Unnamed: 0',axis=1)

In [ ]:
default.columns.shape[0],y_3106_df.columns.shape[0]

In [ ]:
combined_df = pd.concat([default,y_3106_df],ignore_index=True)

In [ ]:
combined_df.to_csv("./default_updated_Y3106.csv")

In [47]:
def calculate_similarity(seq1, seq2):
    alignments = pairwise2.align.globalxx(seq1, seq2)
    best_alignment = alignments[0]
    aligned_seq1, aligned_seq2, score, start, end = best_alignment
    similarity_percentage = score / max(len(seq1), len(seq2)) * 100
    return similarity_percentage

# Filter the second dataframe based on the similarity condition
y_3106_df_filtered = y_3106_df[y_3106_df['sequence'].apply(lambda seq2: all(calculate_similarity(seq2, seq1) <= 30 for seq1 in default['Sequence']))]

# Display the filtered dataframe
print(y_3106_df_filtered)


0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 